# Data Exploration
----

This notebook explores and accesses data from the file based database for buffalo trace

In [1]:
import sqlalchemy 
import pandas as pd 
from importlib import resources
import datetime as dt
from plotly import express as pe

In [3]:
with resources.path("bourbon","buffalo_trace.db") as f: 
    my_conn = sqlalchemy.create_engine(f"sqlite:////{f.absolute()}")

latest_date = my_conn.execute(sqlalchemy.text("select max(update_date) from product_avail")).fetchone()[0]
today_batch = pd.read_sql(
    f"select distinct * from product_avail where update_date='{latest_date}'",
    con=my_conn,
    parse_dates=['update_date']
)
today_batch.astype({"product_title": "string","update_date":"datetime64[ns]","product_available":"boolean"})

,product_title,update_date,product_available
0,Buffalo Trace,2023-02-06 12:43:00,False
1,Eagle Rare,2023-02-06 12:43:00,False
2,E.H. Taylor Jr. Small Batch,2023-02-06 12:43:00,False
3,Blanton’s Single Barrel,2023-02-06 12:43:00,False
4,W.L. Weller Special Reserve,2023-02-06 12:43:00,True
5,Sazerac Rye,2023-02-06 12:43:00,True
6,Wheatley,2023-02-06 12:43:00,True
7,Bourbon Cream,2023-02-06 12:43:00,True


## Clean SQL

In [6]:
my_sql = """select * FROM buffalo_trace pa 
where 
strftime('%Y',update_date) = strftime('%Y',datetime('now'))
and 
strftime('%m',update_date) = strftime('%m',datetime('now'))
;"""

my_data = pd.read_sql(sql=my_sql, con=my_conn, parse_dates=["update_date"])

my_data.head(8)

OperationalError: (sqlite3.OperationalError) no such column: update_date
[SQL: select * FROM buffalo_trace pa 
where 
strftime('%Y',update_date) = strftime('%Y',datetime('now'))
and 
strftime('%m',update_date) = strftime('%m',datetime('now'))
;]
(Background on this error at: https://sqlalche.me/e/14/e3q8)

In [5]:
pe.line(data_frame=my_data, x="update_date",y="product_available",color='product_title')